# Dog VS Cat - Kaggle Competition

- 进入Top20 要求 Loss < 0.04183
- 进入Top10 要求 Loss < 0.03807

In [1]:
import os
import sys
import cv2
import h5py
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from time import time
from datetime import datetime
from tqdm import tqdm
from utils import get_params_count

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from keras.applications import inception_v3, xception, resnet50, vgg16, vgg19
from keras.applications import InceptionV3, Xception, ResNet50, VGG16, VGG19
from keras.layers import Input, Dense, Dropout, Activation, Flatten, Lambda
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.models import Model
from keras.optimizers import SGD

Using TensorFlow backend.


In [2]:
height = 299
labels = np.array([0] * 12500 + [1] * 12500)
train = np.zeros((25000, height, height, 3), dtype=np.uint8)
test = np.zeros((12500, height, height, 3), dtype=np.uint8)

for i in tqdm(range(12500)):
    img = cv2.imread('./train/cat/%s.jpg' % str(i))
    img = cv2.resize(img, (height, height))
    train[i] = img[:, :, ::-1]
    
for i in tqdm(range(12500)):
    img = cv2.imread('./train/dog/%s.jpg' % str(i))
    img = cv2.resize(img, (height, height))
    train[i + 12500] = img[:, :, ::-1]

for i in tqdm(range(12500)):
    img = cv2.imread('./test/%s.jpg' % str(i + 1))
    img = cv2.resize(img, (height, height))
    test[i] = img[:, :, ::-1]
    
print('Training Data Size = %.2f GB' % (sys.getsizeof(train)/1024**3))
print('Testing Data Size = %.2f GB' % (sys.getsizeof(test)/1024**3))

100%|█████████████████████████████████████████████████████| 12500/12500 [00:26<00:00, 467.91it/s]


Training Data Size = 6.24 GB
Testing Data Size = 3.12 GB


In [6]:
def export_gap(MODEL, preprocess=None, batch_size=128):
    x = Input(shape=(height, height, 3))
    x = Lambda(preprocess)(x)
    model = MODEL(include_top=False, input_tensor=x, weights='imagenet', pooling='avg')
    train_gap = model.predict(train, batch_size=batch_size)
    test_gap = model.predict(test, batch_size=batch_size)
    with h5py.File("gap_%s.h5" % MODEL.__name__, 'w') as f:
        f.create_dataset('train', data=train_gap)
        f.create_dataset('test', data=test_gap)

In [4]:
export_gap(InceptionV3, inception_v3.preprocess_input)

In [7]:
export_gap(Xception, xception.preprocess_input, 64)

In [8]:
train_temp = []
test_temp = []
for gapfile in ['gap_InceptionV3.h5', 'gap_Xception.h5']:
    with h5py.File(gapfile, 'r') as f:
        train_temp.append(np.array(f['train']))
        test_temp.append(np.array(f['test']))
train_gap = np.concatenate(train_temp, axis=1)
test_gap = np.concatenate(test_temp, axis=1)

In [9]:
X_train, X_val, y_train, y_val = train_test_split(train_gap, labels, shuffle=True, test_size=0.2, random_state=42)

In [35]:
x = Input(shape=(X_train.shape[1],))
y = Dropout(0.2)(x)
y = Dense(1, activation='sigmoid', kernel_initializer='he_normal', name='classifier')(y)
model_gap = Model(inputs=x, outputs=y, name='GAP')
model_gap.compile(loss='binary_crossentropy', optimizer='adadelta', metrics=['accuracy'])
print('Trainable: %d, Non-Trainable: %d' % get_params_count(model_gap))

Trainable: 4097, Non-Trainable: 0


In [36]:
# Prepare Callbacks for Model Checkpoint, Early Stopping and Tensorboard.
log_name = '/DogVSCat-EP{epoch:02d}-LOSS{val_loss:.4f}.h5'
log_dir = datetime.now().strftime('gap_model_%Y%m%d_%H%M')
if not os.path.exists(log_dir):
    os.mkdir(log_dir)

es = EarlyStopping(monitor='val_loss', patience=20)
mc = ModelCheckpoint(log_dir + log_name, monitor='val_loss', save_best_only=True)
tb = TensorBoard(log_dir=log_dir)

model_gap.fit(x=X_train, y=y_train, batch_size=16, epochs=5, validation_data=(X_val, y_val), callbacks=[es, mc, tb])

# Use all training data.
# model_gap.fit(x=train_gap, y=labels, batch_size=16, shuffle=True, epochs=5)

Epoch 1/5
25000/25000 [==============================] - 4s - loss: 0.0446 - acc: 0.9906     
Epoch 2/5
25000/25000 [==============================] - 4s - loss: 0.0174 - acc: 0.9945     
Epoch 3/5
25000/25000 [==============================] - 4s - loss: 0.0151 - acc: 0.9952     
Epoch 4/5
25000/25000 [==============================] - 4s - loss: 0.0144 - acc: 0.9953     
Epoch 5/5
25000/25000 [==============================] - 4s - loss: 0.0134 - acc: 0.9958     


In [37]:
y_pred = model_gap.predict(test_gap)
y_pred = y_pred.clip(min=0.005, max=0.995)

In [38]:
with open('test.csv', 'w') as f:
    f.writelines('id,label\n')
    for i in range(12500):
        f.writelines(str(i+1) + ',' + str(y_pred[i][0]) + '\n')